In [13]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D


In [14]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [15]:
X_train = train.drop(labels = ['label'],axis=1)
Y_train = train['label']

In [16]:
X_train /= 255.0
test /= 255.0

In [17]:
X_train.shape

(42000, 784)

In [18]:
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [19]:
Y_train = to_categorical(Y_train,num_classes=10)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)

In [22]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5), activation = 'relu', input_shape = (28,28,1) ))

model.add(MaxPool2D())

model.add(Conv2D(filters = 32, kernel_size = (5,5), activation = 'relu' ))

model.add(MaxPool2D())

model.add(Flatten())


model.add(Dense(units = 256, activation = 'relu'))

model.add(Dense(units = 10, activation = 'softmax'))

model.compile(optimizer = 'adam', loss ='categorical_crossentropy', metrics = ['accuracy'])

In [23]:
from keras.preprocessing.image import ImageDataGenerator

In [24]:

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)

datagen.fit(X_train)

In [26]:
model.fit_generator(datagen.flow(X_train,y_train,batch_size = 64),
                   epochs=10, validation_data=(X_test,y_test),
                   verbose=2, steps_per_epoch = X_train.shape[0] // 64)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
 - 34s - loss: 0.3519 - acc: 0.8877 - val_loss: 0.0947 - val_acc: 0.9743
Epoch 2/10
 - 6s - loss: 0.1078 - acc: 0.9666 - val_loss: 0.0570 - val_acc: 0.9810
Epoch 3/10
 - 6s - loss: 0.0797 - acc: 0.9746 - val_loss: 0.0371 - val_acc: 0.9881
Epoch 4/10
 - 6s - loss: 0.0678 - acc: 0.9789 - val_loss: 0.0514 - val_acc: 0.9838
Epoch 5/10
 - 6s - loss: 0.0542 - acc: 0.9829 - val_loss: 0.0342 - val_acc: 0.9888
Epoch 6/10
 - 6s - loss: 0.0494 - acc: 0.9847 - val_loss: 0.0291 - val_acc: 0.9907
Epoch 7/10
 - 6s - loss: 0.0460 - acc: 0.9858 - val_loss: 0.0373 - val_acc: 0.9893
Epoch 8/10
 - 6s - loss: 0.0421 - acc: 0.9861 - val_loss: 0.0350 - val_acc: 0.9893
Epoch 9/10
 - 6s - loss: 0.0406 - acc: 0.9874 - val_loss: 0.0356 - val_acc: 0.9905
Epoch 10/10
 - 6s - loss: 0.0381 - acc: 0.9886 - val_loss: 0.0277 - val_acc: 0.9919


In [27]:
predictions = model.predict(test)

predictions = np.argmax(predictions,axis=1)

predictions = pd.Series(predictions,name='Label')

In [28]:
final_csv = pd.concat([pd.Series(range(1,28001),name = "ImageId"),predictions],axis = 1)

final_csv.to_csv("submit.csv",index=False)